In [1]:
import numpy as np, pandas as pd
import os
import AnOxPePred_funcs as AOf
from importlib import reload

## Load in the partitions

In [2]:
data_path = 'Data'
result_path = 'Result'
AO_p60 = pd.read_csv(os.path.join(data_path, '03_p60_AO_db.csv'), index_col=0)
AO_p70 = pd.read_csv(os.path.join(data_path, '03_p70_AO_db.csv'), index_col=0)
AO_p80 = pd.read_csv(os.path.join(data_path, '03_p80_AO_db.csv'), index_col=0)
AO_p90 = pd.read_csv(os.path.join(data_path, '03_p90_AO_db.csv'), index_col=0)

## Use k-NN to predict antioxidant activity

Decide which k value to use (default k=5). 
k is the number of most identical sequences used to predict with.

In [3]:
reload(AOf)
AO_p60[['Pred_FRS','Pred_Chel']] = AO_p60.apply(lambda x: AOf.kNN_pred(x, AO_p60, k=5), axis=1)
AO_p70[['Pred_FRS','Pred_Chel']] = AO_p70.apply(lambda x: AOf.kNN_pred(x, AO_p70, k=5), axis=1)
AO_p80[['Pred_FRS','Pred_Chel']] = AO_p80.apply(lambda x: AOf.kNN_pred(x, AO_p80, k=5), axis=1)
AO_p90[['Pred_FRS','Pred_Chel']] = AO_p90.apply(lambda x: AOf.kNN_pred(x, AO_p90, k=5), axis=1)

## Performance

Different metrics (Matthew's Correlation Coefficient (MCC), F1 Score (F1) and Area Under a Curve (AUC)) are used to measure k-NNs performance. Custom metrics are calculated with a threshold (the ones to the right) optimized for MCC, while 0.5 metrics use 0.5 as threshold.

In [4]:
y_label, y_pred = AO_p60.loc[:,['FRS','Chelator']].values, AO_p60.loc[:,['Pred_FRS','Pred_Chel']].values
AO_p60_metrics = AOf.calc_metrics(y_label, y_pred, idx='AO_p60')
y_label, y_pred = AO_p70.loc[:,['FRS','Chelator']].values, AO_p70.loc[:,['Pred_FRS','Pred_Chel']].values
AO_p70_metrics = AOf.calc_metrics(y_label, y_pred, idx='AO_p70')
y_label, y_pred = AO_p80.loc[:,['FRS','Chelator']].values, AO_p80.loc[:,['Pred_FRS','Pred_Chel']].values
AO_p80_metrics = AOf.calc_metrics(y_label, y_pred, idx='AO_p80')
y_label, y_pred = AO_p90.loc[:,['FRS','Chelator']].values, AO_p90.loc[:,['Pred_FRS','Pred_Chel']].values
AO_p90_metrics = AOf.calc_metrics(y_label, y_pred, idx='AO_p90')

kNN_result = pd.concat([AO_p60_metrics,AO_p70_metrics,AO_p80_metrics,AO_p90_metrics], axis=0)
kNN_result

,0.5_AUC,0.5_F1,0.5_MCC,0.5_AUC,0.5_F1,0.5_MCC,custom_AUC,custom_F1,custom_MCC,custom_AUC,custom_F1,custom_MCC,FRS_Threshold,CHEL_Threshold
AO_p60,0.644364,0.576983,0.222286,0.541614,0.000000,-0.020957,0.644364,0.576983,0.222286,0.541614,0.101449,0.057448,0.556,0.270
AO_p70,0.747570,0.671024,0.354892,0.647565,0.061224,0.056399,0.747570,0.671024,0.354892,0.647565,0.216216,0.173887,0.556,0.272
AO_p80,0.758959,0.691466,0.397359,0.723253,0.201835,0.205056,0.758959,0.691466,0.397359,0.723253,0.306818,0.261722,0.556,0.248
AO_p90,0.778446,0.709025,0.426609,0.763463,0.277778,0.298999,0.778446,0.629032,0.427603,0.763463,0.377143,0.337047,0.732,0.288


In [5]:
kNN_result.to_csv(os.path.join(result_path, '04_kNN_results.csv'))